# Linear Regression

In this tutorial we will implement a linear regression model. We will also implement a function that splits the available data into a training and a testting part.

## Problem Setting

We will use the Boston Housing Dataset. This dataset contains information collected by the U.S Census Service concerning housing in the city of Boston in the state of Massachusetts in 1978. Our goal is to predict the median value of the houses in a particular town in the city of Boston given its attributes. Check the file ’housing.names’ for more information on the attributes.

In [5]:
import urllib
import pandas as pd
import numpy as np
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
from sklearn.datasets import load_boston

In [68]:
boston=load_boston()
testfile =urllib.request.URLopener()
testfile.retrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.names", "housing.names")
df=pd.DataFrame(boston.data)
df.columns=['crime_rate','res_land_zoned','industry','charles_river','nox','avg_num_rooms','prop_bf_1940','dst_emply_center','rd_highway_idx','tax_rate','stdnt_tchr_ratio','prop_blacks','low_status_pct']
X=boston.data
y=boston.target

In [7]:
df.head(10)

,crime_rate,res_land_zoned,industry,charles_river,nox,avg_num_rooms,prop_bf_1940,dst_emply_center,rd_highway_idx,tax_rate,stdnt_tchr_ratio,prop_blacks,low_status_pct
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10


### Exercise 1

Write the *split_train_test(X,y,split,seed)*, given an instance matrix $X \in \mathbb{R}^{N \times D}$, labels $y \in Y^N$, a split ratio in $[0, 1]$ and a random seed $\in \mathbb{Z}$. Split the dataset in $(split×100)\%$ of the instances for training our model and the rest for testing, i.e. 

$$ \left|X_{\text{train}}\right| = \lceil \text{split} \cdot N \rceil, \qquad |X_{\text{train}}| + |X_{\text{test}}| = N. $$
Make sure you use the given random number generator seed so we all get the same results. The function is supposed to return:

- X_train, y_train: the training instances and labels;
- X_test, y_test: the test instances and labels,

in the same order as was mentioned.

Hint: It may be helpful to use shuffling functionality (e.g. np.random.shuffle).

In [69]:
def split_train_test(X,y,split,seed):
    #label need a reshape to concatenate
    y = y.reshape(y.shape+(1,))
    np.random.seed(seed)
    #concat the label and input to 14 colum
    Full = np.concatenate((X,y), axis=1)
    #shuffle both label and input at the same time
    np.random.shuffle(Full)
    #find the splitting index
    split_index = X.shape[0] - round(X.shape[0] * split)
    
    #split the data
    X_train = Full[:split_index,:-1]
    y_train = Full[:split_index,-1]

    X_test = Full[split_index:,:-1]
    y_test = Full[split_index:,-1]
    return X_train, y_train, X_test, y_test

### Exercise 2

Write the function *train_linear_reg(X_train,y_train,lmbd)*.
Implement the ridge regression model (slide 24). The function should output the learned weight vector $\theta \in \mathbb{R}^D$ or $\mathbb{R}^{D+1}$ (depending on whether you are adding *bias*).

In [197]:
def train_linear_reg(X, y, lmbd):
    #Matrix
    X = np.array(X)
    #Transpose Matrix
    X_t = np.transpose(X)
    #Square Matrix
    XX_t =  X_t.dot(X)
    #Dimension of the Square Matrix
    n = XX_t.shape[0]  
    return np.linalg.inv((XX_t + lmbd * np.identity(n))  ).dot(X_t.dot(y))

### Exercise 3

Write the function *predict(X,theta)* which predicts housing values vector pred for a dataset X and a previously trained parameter vector $\theta$.

In [124]:
def predict(X, theta):
    return X.dot(theta)

### Exercise 4

Write the function *mean_abs_loss(y_true,y_pred)* which computes the mean of the absolute differences between our prediction vector $y\_pred$ and the real housing values $y\_true$.

In [194]:
def mean_abs_loss(y_true,y_pred):     
    return np.mean(abs(y_true -  y_pred))

### Exercise 5

Evaluate your solutions by running the following code. 

Moreover, answer the following questions: What is the most important feature in your model? Are there features that are not so important? What happens if you remove them? Are there outliers with a high absolute loss?

In [239]:
seed = 3
lmbd=1
#bei split 0.1 beste Ergebniss
split=0.1
X_train,y_train,X_test,y_test=split_train_test(X,y,split,seed)
theta=train_linear_reg(X_train,y_train,lmbd)
y_pred=predict(X_test,theta)
mae=mean_abs_loss(y_test,y_pred)
print ('The mean absolute loss is {loss:0.3f}'.format(loss=mae))

The mean absolute loss is 2.672


In [240]:
y_pred=predict(X_test,theta)

In [244]:
list_of_loss = abs(y_pred - y_test)

In [246]:
np.sort(list_of_loss)

array([0.04370312, 0.0530506 , 0.08065609, 0.10203261, 0.17792376,
       0.18102522, 0.30204618, 0.35267713, 0.42160682, 0.4453847 ,
       0.47802173, 0.51947342, 0.61175178, 0.81140599, 0.88470567,
       0.96855488, 0.99855922, 1.26838428, 1.38850641, 1.46861669,
       1.56122124, 1.61824194, 1.74855312, 1.91805465, 1.92027923,
       2.1050764 , 2.1484009 , 2.16025083, 2.31946452, 2.38977842,
       2.39547197, 2.46099017, 2.52992713, 2.59266099, 2.80617245,
       3.07873493, 3.57027832, 4.13659412, 4.20877382, 4.39790042,
       4.87593877, 4.96699205, 5.48972487, 5.49934289, 6.48313334,
       6.74831912, 7.23649412, 7.35951866, 7.62056554, 7.87204346,
       8.50766145])

In [218]:
from sklearn.linear_model import LinearRegression

In [223]:
model = LinearRegression().fit(X_train, y_train)

In [224]:
y_pred = model.predict(X_test)

In [225]:
mean_abs_loss(y_test,y_pred)

3.6487407809516634